In [1]:
using Plots
using LaTeXStrings
using FilePathsBase
using Dates

In [2]:
include("src/constants.jl")
include("src/main.jl")
include("src/solver_code.jl")
using .MainModule # モジュールを使用
using .SolverCode

# Typical scale
println("length scale: ", unit_l, " cm")
println("energy density scale: ", eps_ref, " g/cm^3")
println("energy density scale: ", eps_ref*gcm3_to_MeVfm3/1000, " GeV/fm^3")

length scale: 1.0 cm
energy density scale: 1.2102591909632933e49 g/cm^3
energy density scale: 6.789056165277286e33 GeV/fm^3


In [3]:
# ファイルパスを指定
file_path = "data/eos_HQC18_gv100H160.txt"
P = []
e = []
cs2 = [] 
# ファイルを行単位で読み込む
lines = readlines(file_path)
println(lines[1])
for line in lines[2:end]
    # 行をスペースで分割して各値を数値に変換
    values = parse.(Float64, split(line))
    push!(P, values[2]) 
    push!(e, values[4]) 
    push!(cs2, values[5].^2) 
end

file_path2 = "data/eos_HQC18_gv080H150.txt"
P2 = []
e2 = []
lines = readlines(file_path2)
for line in lines[2:end]
    values = parse.(Float64, split(line))
    push!(P2, values[2]) 
    push!(e2, values[4])
end

file_path3 = "data/eos_HQC18_gv050H140.txt"
P3 = []
e3 = []
lines = readlines(file_path3)
for line in lines[2:end]
    values = parse.(Float64, split(line))
    push!(P3, values[2]) 
    push!(e3, values[4])
end

 muB (MeV)    P (MeV/fm^3)           nB/n0    e (MeV/fm^3)              cs


In [4]:
mono_e, mono_P = MainModule.make_eos_monotonic(e, P)
mono_e2, mono_P2 = MainModule.make_eos_monotonic(e2, P2)
mono_e3, mono_P3 = MainModule.make_eos_monotonic(e3, P3)

println(length(mono_e))
# グラフの描画
plot(mono_e, mono_P;
    label="eos_HQC18_gv100H160", 
    xlabel=L"\mathrm{Energy\ density\ [MeV/fm^3]}",xlabelfontsize=16,
    ylabel=L"\mathrm{Pressure\ [MeV/fm^3]}", ylabelfontsize=16,
    title="Eos",
    legend=:topleft,
    xscale=:log10,  # x軸を対数スケールに変更
    yscale=:log10,  # y軸を対数スケールに変更)
    xtickfontsize=12,
    ytickfontsize=12,
    xlims=(1,2000),
    ylims=(1e-3, 1000),
)
plot!(mono_e2, mono_P2; label=splitext(basename(file_path2))[1])
plot!(mono_e3, mono_P3; label=splitext(basename(file_path3))[1])
# 凡例のフォントサイズを変更
plot!(legendfontsize=11)  # 凡例のフォントサイズを指定
savefig("three_types_QHC18_eos.pdf")  # PDFファイルとして保存

386


"/Users/mnmgchjsk/JupyterLab/TOV_solver_Julia/three_types_QHC18_eos.pdf"

In [5]:
plot(mono_e[2:end],
     [(mono_P[i]-mono_P[i-1])/(mono_e[i]-mono_e[i-1]) for i in 2:length(mono_P)];
     label="eos_HQC18_gv100H160", 
     xlabel=L"\mathrm{Energy\ density\ [MeV/fm^3]}",xlabelfontsize=16,
     ylabel=L"\mathrm{c_s^2}", ylabelfontsize=16,
     title="Speed of sound",
     legend=:topleft,
     xscale=:log10,  # x軸を対数スケールに変更
     yscale=:identity,
     xlims=(100,mono_e[end]),
     xtickfontsize=12,
     ytickfontsize=12,
)

plot!(mono_e2[2:end],
     [(mono_P2[i]-mono_P2[i-1])/(mono_e2[i]-mono_e2[i-1]) for i in 2:length(mono_P2)];
     label=splitext(basename(file_path2))[1])

plot!(mono_e3[2:end],
     [(mono_P3[i]-mono_P3[i-1])/(mono_e3[i]-mono_e3[i-1]) for i in 2:length(mono_P3)];
     label=splitext(basename(file_path3))[1])
# 凡例のフォントサイズを変更
plot!(legendfontsize=11)  # 凡例のフォントサイズを指定
savefig("three_types_QHC18_cs2.pdf")  # PDFファイルとして保存

"/Users/mnmgchjsk/JupyterLab/TOV_solver_Julia/three_types_QHC18_cs2.pdf"

In [6]:
start_time = now()
RMT, sol = MainModule.out_RMT(mono_e*MeVfm3_to_gcm3/unit_g, mono_P*MeVfm3_to_gcm3/unit_g, debug=false)
RMT2, sol2 = MainModule.out_RMT(mono_e2*MeVfm3_to_gcm3/unit_g, mono_P2*MeVfm3_to_gcm3/unit_g)
RMT3, sol3 = MainModule.out_RMT(mono_e3*MeVfm3_to_gcm3/unit_g, mono_P3*MeVfm3_to_gcm3/unit_g)
end_time = now()

elapsed_ms = Dates.value(end_time - start_time)  # ミリ秒で取得
elapsed_sec = elapsed_ms / 3000  # 秒に変換
println(elapsed_sec, " sec/RMT")

3.287333333333333 sec/RMT


In [7]:
plot(RMT[1], RMT[2]; 
    label=splitext(basename(file_path))[1],
    legend=:topright,
    xlabel=L"R\ [\mathrm{km}]", xlabelfontsize=16,
    ylabel=L"M\ [M_\odot]", ylabelfontsize=16,
    xlims=(8,16),
    xtickfontsize=12,
    ytickfontsize=12,
)
plot!(RMT2[1], RMT2[2]; label=splitext(basename(file_path2))[1])
plot!(RMT3[1], RMT3[2]; label=splitext(basename(file_path3))[1])
# 凡例のフォントサイズを変更
plot!(legendfontsize=11)  # 凡例のフォントサイズを指定
savefig("three_types_QHC18_MR.pdf")  # PDFファイルとして保存

"/Users/mnmgchjsk/JupyterLab/TOV_solver_Julia/three_types_QHC18_MR.pdf"

In [8]:
plot(RMT[3], RMT[2];
    label=splitext(basename(file_path))[1],
    xscale=:log10,
    xlabel=L"\Lambda", xlabelfontsize=16,
    xticks=[10^-2, 10^-1, 1],
    ylabel=L"M\ [\mathrm{M_\odot}]", ylabelfontsize=16,
    xtickfontsize=12,
    ytickfontsize=12,
)
plot!(RMT2[3], RMT2[2]; label=splitext(basename(file_path2))[1])
plot!(RMT3[3], RMT3[2]; label=splitext(basename(file_path3))[1])
# 凡例のフォントサイズを変更
plot!(legendfontsize=11)  # 凡例のフォントサイズを指定
savefig("three_types_QHC18_MT.pdf")  # PDFファイルとして保存

"/Users/mnmgchjsk/JupyterLab/TOV_solver_Julia/three_types_QHC18_MT.pdf"